In [ ]:
#
#    Notebook de cours MAP412 - Chapitre 8 - M. Massot 2022-2023 - Ecole polytechnique
#    ----------   
#    Challenge : Stabilité forward
#    
#    Auteurs : L. Séries et M. Massot - (C) 2022
#

# Challenge : Analyse d'erreur

In [ ]:
import numpy as np
from mpmath import mp
import mpmath
import plotly.graph_objects as go
from plotly.subplots import make_subplots

## Méthode d'Euler explicite à précision fixée

In [ ]:
#####################################################
class ode_result:
    def __init__(self, y, t):
        self.y = y
        self.t = t

#####################################################
def forward_euler(tini, tend, nt, yini, fcn):

    dt = (tend-tini) / (nt-1)
    t = np.linspace(tini, tend, nt)

    yini = np.atleast_1d(yini)
    neq = yini.size

    y = np.zeros((neq, nt))
    y[:,0] = yini

    for it, tn  in enumerate(t[:-1]):
        yn = y[:,it]
        y[:,it+1] = yn + dt*np.array(fcn(tn, yn))

    return ode_result(y, t)

#####################################################
def forward_euler_mp(tini, tend, nt, yini, fcn):

    dt = (tend-tini) / (nt-1)
    
    y = np.array(mpmath.zeros(int(nt),1))
    
    y[0] = yini

    tn = tini
    
    for it in range(nt-1):
        yn = y[it]
        y[it+1] = yn + dt*fcn(tn, yn)
        tn = tn + dt

    return y

### Résultats sur le problème Curtiss et Hirschfelder

In [ ]:
class curtiss_model:

    def __init__(self, k):
        self.k = k

    def fcn(self, t, y) :
        k = self.k
        cos = np.frompyfunc(mpmath.cos, 1, 1) 
        y_dot = k * (cos(t) - y)
        return y_dot

    def sol(self, yini, t0, t):
        k = self.k
        cos = np.frompyfunc(mpmath.cos, 1, 1) 
        sin = np.frompyfunc(mpmath.sin, 1, 1) 
        exp = np.frompyfunc(mpmath.exp, 1, 1) 
        c0 = (yini - (k/(k*k + 1)) * (k*cos(t0) + sin(t0))) * exp(-k*t0)
        y = (k/(k*k + 1)) * (k*cos(t) + sin(t)) +  c0 * exp(-k*t)
        return y

### Evaluation de la constante 

In [ ]:
yini = 1.
tini = 0.
tend = 1.5
cm = curtiss_model(k=50)
fcn = cm.fcn

nt = np.array([151, 1501, 15001, 150001])

err_inf = np.zeros_like(nt, dtype=float)

for i, nt_i in enumerate(nt):
    sol = forward_euler(tini, tend, nt_i, yini, fcn)
    yerr = np.abs(cm.sol(yini, tini, sol.t) - sol.y[0])
    err_inf[i] = np.linalg.norm(yerr,np.inf) 

dt = (tend-tini)/(nt-1)

fig = go.Figure()
fig.add_trace(go.Scatter(x=dt, y=err_inf, name='Norme linf'))
fig.add_trace(go.Scatter(x=dt, y=0.01*dt, mode='lines', line_dash='dot', name='pente 1'))
fig.update_xaxes(type='log', exponentformat='e', title='dt')
fig.update_yaxes(type='log', exponentformat='e', title="Norme de l'erreur")
fig.show()

## Solution à précision fixée

In [ ]:
yini_ref = mp.mpf('1.')
tini_ref = mp.mpf('0.')
tend_ref = mp.mpf('1.5')
k_ref    = mp.mpf('50.')
cm_ref   = curtiss_model(k_ref)

mp01 = mp.clone()
mp01.prec = 20
yini = mp01.mpf('1.') 
tini = mp01.mpf('0.')
tend = mp01.mpf('1.5')
k    = mp01.mpf('50.')
cm   = curtiss_model(k)

print(f"Nombre de chiffres significatifs : {mp01.dps}")

fig = make_subplots(rows=2, cols=1, subplot_titles=("Solution", "Erreur globale"), vertical_spacing=0.1)

nt = np.array([100, 150, 200, 305, 400, 500, 600, 700, 1000, 2000, 5000, 10000, 20000, 80000])

for nt_i in nt: 

    texa = mp.linspace(tini_ref, tend_ref, nt_i)
    texa = np.array(texa)
    yexa = cm_ref.sol(yini_ref, tini_ref, texa)

    ysol = forward_euler_mp(tini, tend, nt_i, yini, cm.fcn)

    err = np.abs(ysol-yexa)

    fig.add_trace(go.Scattergl(visible=False, x=np.array(texa, dtype=float), y=np.array(yexa, dtype=float), name='Sol. exacte'), row=1, col=1)
    fig.add_trace(go.Scattergl(visible=False, x=np.array(texa, dtype=float), y=np.array(ysol, dtype=float), name='Sol. num.'), row=1, col=1)
    fig.add_trace(go.Scattergl(visible=False, x=np.array(texa, dtype=float), y=np.array(err, dtype=float), name='erreur'), row=2, col=1)

fig.data[0].visible = True
fig.data[1].visible = True
fig.data[2].visible = True
    
# create and add slider
steps = []
for i, nt_i in enumerate(nt):
    step = dict(method="update", label = f" {nt_i}",
                args=[{"visible": [el==3*i or el==3*i+1 or el==3*i+2  for el in range(len(fig.data))]}])
    steps.append(step)
        
sliders = [dict(currentvalue={"prefix": "nb points: "}, steps=steps)]

fig.update_yaxes(exponentformat='e', row=2)
fig.update_layout(height=800, sliders=sliders)

Proposer une explication du comportement de l'erreur quand le pas de temps diminue.